**imports**

In [14]:
import pandas as pd
import random

# Code

In [15]:
#get dir
outputs_dir = "./model_outputs/"

#set random seed - reproducibility
random.seed(213)

In [17]:
#open test data
test_data = pd.read_csv("./data/test_data.csv")

#get 26 random indices from test data length (around 1% of data)
random_indices = random.sample(range(len(test_data)), 26)

#file names (why not hard code in lol)
names = ["generated_output_gru.csv", "generated_output_gru_non_deterministic.csv", "generated_output_seq2seq.csv",
         "generated_output_seq2seq_non_deterministic.csv", "generated_output_t5.csv", "generated_output_t5_non_deterministic.csv",
         "generated_output_t5_all.csv", "generated_output_t5_all_non_deterministic.csv"]

#open each file as df and store in list
dataframes = [pd.read_csv(outputs_dir + name) for name in names]

#collect reframed thoughts from each file using the same indices
collected_texts = [] #init

for dataframe in dataframes:
    selected = dataframe.loc[random_indices, "Generated_Text"].reset_index(drop=True)
    collected_texts.append(selected)

#combine into one dataframe with one column per model
generated_text_df = pd.concat(collected_texts, axis=1)
generated_text_df.columns = [name[17:-4]for name in names] #remove "generated_output_" and ".csv"

#get negative and reframed thoughts to put in front of csv file (compare to outputs)
control_columns = test_data.loc[random_indices, ["negative_thought", "reframed_thought"]].reset_index(drop=True)

#add to front of dataframe
final_dataframe = pd.concat([control_columns, generated_text_df], axis=1)

#save to csv
final_dataframe.to_csv("combined_generated_outputs.csv", index=False)